In [ ]:
pip install transformers==2.6.0

In [2]:
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm, trange


import pickle
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import transformers
from transformers import BertForTokenClassification, AdamW


In [3]:
import keras_preprocessing
keras_preprocessing.__version__



'1.1.2'

In [5]:
class nerModel:
    
  def __init__(self, model_path, idx2tag_path):
    self.ner_model = {}
    self.idx2tag = pickle.load(open(os.path.join(idx2tag_path, "idx2tag.pkl"), 'rb'))
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=True)
    if torch.cuda.is_available():
      self.model = torch.load(os.path.join(model_path,"model.pt"))
    else:
      self.model = torch.load(os.path.join(model_path,"model.pt"), map_location=torch.device('cpu'))
    # self.model = torch.load(os.path.join(model_path,"model.pt"))
    self.model.eval()
         
    
  def do_pridict(self, input_sentence):
    # first toknize the sentences
    tokenized_sentence = self.tokenizer.encode(input_sentence)
    if torch.cuda.is_available():
      input_ids = torch.tensor([tokenized_sentence]).cuda()
    else:
      input_ids = torch.tensor([tokenized_sentence])
    # input_ids = torch.tensor([tokenized_sentence]).cuda()
    # run the sentences through the model
    with torch.no_grad():
        output = self.model(input_ids)
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)


    # join bpe split tokens
    tokens = self.tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
    new_tokens, new_labels = [], []
    for token, label_idx in zip(tokens, label_indices[0]):
        if token.startswith("##"):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_labels.append(self.idx2tag[label_idx])
            print(label_idx)
            new_tokens.append(token)
    return new_tokens, new_labels, output

  def predict_proba(self, input_sentence):
    # first toknize the sentences
    tokenized_sentence = self.tokenizer.encode(input_sentence)
    input_ids = torch.tensor([tokenized_sentence])
    # run the sentences through the model
    with torch.no_grad():
        output = self.model(input_ids)
    pred = output[0][0].to('cpu').numpy()

    return np.array(pred)
      


In [7]:
# Creat an object for nerModel class and pass the path toner saved model.pt and idx2tag dictionary
ner_obj = nerModel('/content/model', '/content/asset')

In [8]:
test_sentence = """
Kedir is an engineer work at Amazon.
"""

In [9]:
tokens, labels, output = ner_obj.do_pridict(test_sentence)



12
11
12
12
12
12
12
9
12
12


In [26]:
for token, label in zip(tokens, labels):
  # if token not in ['[CLS]','[SEP]']:
    print("{}\t{}".format(label, token))

O	[CLS]
B-per	kedir
O	is
O	an
O	engineer
O	work
O	at
B-org	amazon
O	.
O	[SEP]
